In [30]:
# 导入库 Importing libraries
import cv2
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
import time
# 线程功能操作库 Thread function operation library
import threading
import inspect
import ctypes
import HSV_Config
#显示摄像头组件 Display camera components
image_widget = widgets.Image(format='jpeg', width=320, height=240)

In [31]:
import os,socket,sys,time
import spidev as SPI
import xgoscreen.LCD_2inch as LCD_2inch
from PIL import Image,ImageDraw,ImageFont
import numpy as np
import mediapipe as mp
from numpy import linalg
from xgolib import XGO

g_dog = XGO(port='/dev/ttyAMA0',version="xgolite")

In [32]:
#屏幕清除
mydisplay = LCD_2inch.LCD_2inch()
mydisplay.clear()
splash = Image.new("RGB", (mydisplay.height, mydisplay.width ),"black")
mydisplay.ShowImage(splash)

In [33]:
#bgr8转jpeg格式  bgr8 to jpeg format
import enum
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [34]:
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [35]:
#USB摄像头打开方式 USB camera opening method

image = cv2.VideoCapture(0)  #打开摄像头/dev/video0 Open the camera /dev/video0
image.set(3, 320)
image.set(4, 240)
image.set(5, 30)  #设置帧率 Setting the frame rate
# image.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
# image.set(cv2.CAP_PROP_BRIGHTNESS, 62) #设置亮度 -64 - 64  0.0 Set Brightness -64 - 64 0.0
# image.set(cv2.CAP_PROP_CONTRAST, 63) #设置对比度 -64 - 64  2.0 Set Contrast -64 - 64 2.0
# image.set(cv2.CAP_PROP_EXPOSURE, 4800) #设置曝光值 1.0 - 5000  156.0 Set the exposure value 1.0 - 5000 156.0

ret, frame = image.read()
image_widget.value = bgr8_to_jpeg(frame)
update_hsv = HSV_Config.update_hsv()


### HSV值存储

In [36]:
color_hsv  = {"red"   : ((0, 70, 72), (7, 255, 255)),
              "green" : ((54, 109, 78), (77, 255, 255)),
              "blue"  : ((92, 100, 62), (121, 251, 255)),
              "yellow": ((26, 100, 91), (32, 255, 255))}

In [37]:
def TEST():
    detected_colors = []  # 用于存储检测到的颜色名称
    unique_colorstemp=None
    global color_hsv
    display(image_widget)
    while True:
        ret, frame = image.read() #usb摄像头 usb camera
        frame, binary,hsvname=update_hsv.get_contours(frame,color_hsv)
        unique_colors = list(dict.fromkeys(hsvname))
        # 根据列表的长度来决定如何分割字符串
        num_colors = len(unique_colors)
       
        if(unique_colors!=unique_colorstemp):
            unique_colorstemp=unique_colors.copy()
        
        # 实时传回图像数据进行显示 Real-time image data transmission for display
        image_widget.value = bgr8_to_jpeg(frame)

        #显示在小车的lcd屏幕上
        b,g,r = cv2.split(frame)
        img = cv2.merge((r,g,b))
        imgok = Image.fromarray(img)
        mydisplay.ShowImage(imgok)
        

In [38]:
thread1 = threading.Thread(target=TEST)
thread1.daemon = True
thread1.start()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [39]:
stop_thread(thread1)

In [40]:
#usb摄像头 usb camera
#使用完成对象记住释放掉对象，不然下一个程序使用这个对象模块会被占用，导致无法使用
#After using the object, remember to release the object, otherwise the object module will be occupied by the next program, making it unusable
image.release()